<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/it_whisper_large_test_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-2m4kbbjl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-2m4kbbjl
  Resolved https://github.com/huggingface/transformers to commit 58e7f9bb2faf30622c9bead7adf472ac59f3d301
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8178366 sha256=a03eb869ab1bd9e3b64f88979befac1e62b216060eebae84edb73c0d1aed92e4
  Stored in directory: /tmp/pip-ephem-wheel-cache-bxcs3bs3/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "it", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1177
})

In [6]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 11.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=6a44e2a44aadb119cbfc8573a23ebcd3e133464fc3b189ccb4c324e91a9f3b8d
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [7]:
model = whisper.load_model("large")

100%|██████████████████████████████████████| 2.88G/2.88G [00:27<00:00, 113MiB/s]


In [8]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [9]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [10]:
text

,0
0,questo è fondamentale se pensiamo a paesi come...
1,allora punto numero uno inserire l'impegno del...
2,qui naturalmente c'è chi ha la verità in tasca...
3,il g otto e il g venti delle settimane scorse ...
4,signora presidente signor commissario potonik ...
...,...
1172,signor presidente soltanto un'informazione per...
1173,per fermare queste contrapposizioni che si svo...
1174,siamo quasi in un regime di dittatura europea ...
1175,signor presidente onorevoli colleghi nella rel...


In [11]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [12]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,questo è fondamentale se pensiamo a paesi come...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0203857421875, -0.037109375, 0.000793457031...",16000
1,female,allora punto numero uno inserire l'impegno del...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.013397216796875, 0.02423095703125, 0.036041...",16000
2,male,qui naturalmente c'è chi ha la verità in tasca...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0390625, 0.035186767578125, 0.0305786132812...",16000
3,male,il g otto e il g venti delle settimane scorse ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 9.1552734375e-05, -6.10351...",16000
4,male,signora presidente signor commissario potonik ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000640869140625, 0.000457763671875, -0.0014...",16000
...,...,...,...,...,...
1172,male,signor presidente soltanto un'informazione per...,/root/.cache/huggingface/datasets/downloads/ex...,"[-9.1552734375e-05, -0.000518798828125, -0.005...",16000
1173,male,per fermare queste contrapposizioni che si svo...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00311279296875, 0.003936767578125, 0.003112...",16000
1174,male,siamo quasi in un regime di dittatura europea ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.02105712890625, 0.024627685546875, 0.033691...",16000
1175,male,signor presidente onorevoli colleghi nella rel...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004058837890625, 0.004150390625, 0.00357055...",16000


In [13]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,questo è fondamentale se pensiamo a paesi come...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0203857421875, -0.037109375, 0.000793457031...",16000
2,male,qui naturalmente c'è chi ha la verità in tasca...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0390625, 0.035186767578125, 0.0305786132812...",16000
3,male,il g otto e il g venti delle settimane scorse ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 9.1552734375e-05, -6.10351...",16000
4,male,signora presidente signor commissario potonik ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000640869140625, 0.000457763671875, -0.0014...",16000
5,male,la restituzione dei beni rubati deve essere in...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001953125, -0.003204345703125, 0.0010375976...",16000
...,...,...,...,...,...
1172,male,signor presidente soltanto un'informazione per...,/root/.cache/huggingface/datasets/downloads/ex...,"[-9.1552734375e-05, -0.000518798828125, -0.005...",16000
1173,male,per fermare queste contrapposizioni che si svo...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00311279296875, 0.003936767578125, 0.003112...",16000
1174,male,siamo quasi in un regime di dittatura europea ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.02105712890625, 0.024627685546875, 0.033691...",16000
1175,male,signor presidente onorevoli colleghi nella rel...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004058837890625, 0.004150390625, 0.00357055...",16000


In [14]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
1,female,allora punto numero uno inserire l'impegno del...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.013397216796875, 0.02423095703125, 0.036041...",16000
11,female,quindi problemi sul piano della sicurezza sul ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001220703125, -0.001068115234375, -0.00131...",16000
12,female,questo è vero per i cristiani in medio oriente...,/root/.cache/huggingface/datasets/downloads/ex...,"[3.0517578125e-05, -0.000640869140625, -0.0004...",16000
14,female,di una guerra per procura sulla libia e in lib...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000213623046875, -0.000823974609375, -0.00...",16000
18,female,ricordo che tra gli obiettivi più ampi delle n...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003387451171875, -0.001373291015625, -0.00...",16000
...,...,...,...,...,...
1145,female,scarse informazioni per i passeggeri assistenz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00341796875, -0.001953125, -0.002227783203...",16000
1149,female,signora presidente onorevoli colleghi ringrazi...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000823974609375, -0.000457763671875, -0.000...",16000
1152,female,la libia ha grandissime capacità quello che l'...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0462646484375, 0.049285888671875, 0.0446777...",16000
1165,female,la necessità di proteggere tutte le comunità e...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.10955810546875, -0.15838623046875, -0.2659...",16000


In [15]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="it")
    return result["text"]

In [16]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [17]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.9696969696969697
Processed 2 files, WER: 0.25806451612903225
Processed 3 files, WER: 0.1590909090909091
Processed 4 files, WER: 0.23809523809523808
Processed 5 files, WER: 0.975
Processed 6 files, WER: 1.0
Processed 7 files, WER: 0.25
Processed 8 files, WER: 0.1111111111111111
Processed 9 files, WER: 0.875
Processed 10 files, WER: 0.9565217391304348
Processed 11 files, WER: 0.5555555555555556
Processed 12 files, WER: 0.06060606060606061
Processed 13 files, WER: 1.25
Processed 14 files, WER: 0.38095238095238093
Processed 15 files, WER: 0.09090909090909091
Processed 16 files, WER: 0.875
Processed 17 files, WER: 0.14705882352941177
Processed 18 files, WER: 0.20408163265306123
Processed 19 files, WER: 0.047619047619047616
Processed 20 files, WER: 0.36585365853658536
Processed 21 files, WER: 0.21739130434782608
Processed 22 files, WER: 1.6666666666666667
Processed 23 files, WER: 0.21428571428571427
Processed 24 files, WER: 0.3333333333333333
Processed 25 files, WER

In [18]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.3783783783783784
Processed 2 files, WER: 0.3958333333333333
Processed 3 files, WER: 0.2222222222222222
Processed 4 files, WER: 0.2608695652173913
Processed 5 files, WER: 0.19444444444444445
Processed 6 files, WER: 0.3
Processed 7 files, WER: 0.9285714285714286
Processed 8 files, WER: 0.425
Processed 9 files, WER: 0.29411764705882354
Processed 10 files, WER: 0.11764705882352941
Processed 11 files, WER: 0.9772727272727273
Processed 12 files, WER: 0.14285714285714285
Processed 13 files, WER: 0.9714285714285714
Processed 14 files, WER: 0.03333333333333333
Processed 15 files, WER: 0.2127659574468085
Processed 16 files, WER: 0.058823529411764705
Processed 17 files, WER: 0.2631578947368421
Processed 18 files, WER: 1.0
Processed 19 files, WER: 1.0
Processed 20 files, WER: 0.288135593220339
Processed 21 files, WER: 0.22857142857142856
Processed 22 files, WER: 1.0
Processed 23 files, WER: 0.2857142857142857
Processed 24 files, WER: 0.08
Processed 25 files, WER: 0.272727